# The Data Scientist's Notebook

Only data privacy is protected in this use case. The model created by the data scientist is not kept private since it is sent to the data owner's machine for training without being encrypted.

**Note:**

Much of the code used here is either copied or adapted from the `Word-level language modeling` PyTorch example:

https://github.com/pytorch/examples/tree/master/word_language_model

The goal being to demonstrate how the original example could be adapted to a context where the dataset is private to the data owner as it is the case in this demo.

## PART 0: Connect to a Remote Duet Server

In [1]:
import syft as sy
import torch

Before connecting to the remote duet server, the data owner should first launch a duet server. After launch, the data scientist can connect to the duet server.

In [2]:
duet = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ joining duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently 
♫♫♫ > in alpha. Do not use this to protect real-world data.
♫♫♫ >
♫♫♫ > Punching through firewall to OpenGrid Network Node at: http://ec2-18-216-8-163.us-east-2.compute.amazonaws.com:5000
♫♫♫ > http://ec2-18-216-8-163.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... DONE!

♫♫♫ > Duet Client ID: 7a64251b8d2bfec49c42d935edd87ae2

♫♫♫ > STEP 1: Send the Duet Client ID to your duet partner!

♫♫♫ > ...waiting for partner to connect...
♫♫♫ > ...using a running event loop...

♫♫♫ > CONNECTED!


2020-11-10 16:40:28.035 | ERROR    | syft.grid.duet.webrtc_duet:pull:250 - Got an exception in Duet pull. 
2020-11-10 16:40:31.117 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 16:40:36.120 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 16:40:46.123 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 16:40:51.125 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


## PART 1: Get Pointers to Shared Objects

Get a list of the shared objects:

In [3]:
duet.store.pandas

,ID,Tags,Description
0,<UID:ff36e738-8dd3-4d51-bc7c-fbe75acdc63a>,"[wikitext2_dataset, vocab_size]",Vocabulary size of Wikitext2 dataset
1,<UID:7638c989-026b-4300-8608-e6bec01a0fc3>,"[wikitext2_dataset, train_data]","Wikitext2 training set. shape: (2088628,)"


2020-11-10 16:40:56.127 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Get the size of the dataset's vocabulary

In [4]:
vocab_size = duet.store[0]
vocab_size = vocab_size.get_copy(request_block = True)
vocab_size = int(vocab_size)
vocab_size


> Waiting for Blocking Request
<UID:0d823035-1f94-450e-a825-1893d83e6bd8>

> INSIDE Request BLOCK 9.5367431640625e-07 seconds False
.
> INSIDE Request BLOCK 1.069579839706421 seconds False

> Blocking Request ACCEPTED


33278

In [5]:
vocab_size

33278

2020-11-10 16:41:01.130 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Get references to the datasets

In [6]:
train_data = duet.store[1]
#valid_set = duet.store[2]
#train_set = duet.store['22d54a82-b7e7-40da-adfb-06a588121ba1']

2020-11-10 16:41:06.134 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


## PART 2: Prepare Datasets for Training

The training and validation sets, as shared by the data owners, are flat tensors of the form:

```
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16 .....]
```

where the integers represent words.

The data scientist here has the responsiblity of batchifying this dataset to serve training. I avoid giving the DO this responsiblity as I suppose that it is up to the DS to decide how data is batchified.

I suggest to reshape in the following way:

1. Reshape into a list of input/target samples:

```
[ 
  [ [1 , 2 , 3 , 4 ],
    [5 , 6 , 7 , 8 ]  ],
    
  [ [9 , 10, 11, 12], 
    [13, 14, 15, 16]  ],
  :
  :
  :
]
```

This should use the `view()` method in `torch`.

2. Create a `Dataloader` object in the DS side that batchified this training set. For example, for a batch size of 2, the data loader should return:

```
Input batch:
     [ [1 , 2 , 3 , 4 ],
       [9 , 10, 11, 12]  ]
       
Target batch:
     [ [5 , 6 , 7 , 8 ],
       [13, 14, 15, 16]  ]
```

Of course all operations are carried out on tensor pointers because the dataset does not quit the DO's node.

### Build a Dataset class

Fix some hyperparameters:

In [7]:
ntokens_train = 2088628

bsz = 4
bptt = 32
#dropout = 0.5

#
ninp = 100

# Size of hidden layer
nhid = 200

# Number of RNN layer
nlayers =2

In [26]:
class Wikitext2(torch.utils.data.Dataset):
    
    def __init__(self, tokens, ntokens, bsz, bptt):
        
        # A pointer to the tensor that contains the list of 
        # all token IDs in the dataset
        self.tokens = tokens
        
        # The sequence length
        self.bptt = bptt
        
        # The batch size
        self.bsz = bsz
        
        # Number of tokens in the dataset
        self.ntokens = ntokens
        
        
        # Batchify the dataset
        self._batchify()
        
    def __getitem__(self, index):
        
        input, target = self._get_batch(index)        
        
        return input, target

        
    def __len__(self):
                
        return (self.ntokens // self.bsz) - (2 * self.bptt)
    
    
    def _batchify(self):  
        
        # Since we are going to reshape the self.tokens 1D tensor 
        # into a 2D tensor with a number of rows equal to the
        # batch size, we should compute the number of columns
        # of that reshaped tensor
        width = self.ntokens // self.bsz

        # remove surplus tokens
        self.tokens_2d = self.tokens.narrow(0, 0, self.bsz * width)

        # Reshape
        self.tokens_2d = self.tokens_2d.view(self.bsz, -1)
        
        
    def _get_batch(self, index):
        
        input = self.tokens_2d.narrow(dim = 1, start = index, length = self.bptt)
        target = self.tokens_2d.narrow(dim = 1, start = index + self.bptt, length = self.bptt)

        return input, target
        

2020-11-10 15:36:46.474 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [27]:
train_set = Wikitext2(tokens = train_data, 
                      ntokens = ntokens_train, 
                      bsz = bsz,
                      bptt = bptt)

2020-11-10 15:36:51.476 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 15:36:56.478 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 15:37:01.480 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 15:37:06.482 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [97]:
train_loader = torch.utils.data.DataLoader(dataset=train_set, 
                                           batch_size=1, # Should be always set to 1
                                           num_workers=0, 
                                           drop_last=True,
                                           shuffle=True,
                                          )

## PART 3: Built an RNN-based model

Get a pointer to the remote torch and its modules

In [11]:
torch_do = duet.torch
nn = torch_do.nn
F = torch_do.nn.functional

Create the model

In [41]:
class RNNModel(sy.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        self.ntoken = ntoken
        #self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        # Optionally tie weights as in:
        # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
        # https://arxiv.org/abs/1608.05859
        # and
        # "Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling" (Inan et al. 2016)
        # https://arxiv.org/abs/1611.01462
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        #self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.weight)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, x):
        input, hidden = x
        #emb = self.drop(self.encoder(input))
        emb = self.encoder(input)

        result = self.rnn(emb)
        output, hidden = result[0], result[1]
        
        #output = self.drop(output)
        decoded = self.decoder(output)
        decoded = decoded.view(-1, self.ntoken)
        
        return F.log_softmax(decoded, dim=1)#, hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)

Create a model instance

In [42]:
model = RNNModel(rnn_type = 'LSTM', ninp = 100, ntoken = vocab_size, nhid = 100, nlayers =2)

In [43]:
#import torch

#input = torch.ones(2,20, dtype = torch.long).send(duet) # bsz * bptt
#hidden = torch.zeros(2, 2, 20, 100).send(duet) # nb_layers * bptt * nhid
#c = torch.zeros(2, 20, 100).send(duet) # nb_layers * bptt * nhid

2020-11-10 16:49:48.515 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [44]:
#model((input, torch.zeros(1,20)))
output = model((input, None ))


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [46]:
output[0]

2020-11-10 16:50:03.522 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 16:50:08.525 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 16:50:13.528 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-10 23:48:41.587 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-10 23:48:46.591 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


In [38]:
a = output.get(request_block=True)


> Waiting for Blocking Request
<UID:edc66eba-49b5-4c9c-850c-4610dc1112d0>

> INSIDE Request BLOCK 1.1920928955078125e-06 seconds False
.
> INSIDE Request BLOCK 1.0869884490966797 seconds False

> Blocking Request ACCEPTED


2020-11-10 16:48:07.532 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 6


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 6


2020-11-10 16:48:12.535 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 6


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 6


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 6


2020-11-10 16:48:42.585 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 6


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 6


2020-11-10 16:49:08.498 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [39]:
type(a)

torch.Tensor

In [40]:
a[0]

tensor([-10.3951, -10.4903, -10.3292,  ..., -10.5370, -10.4807, -10.3968],
       grad_fn=<SelectBackward>)

2020-11-10 16:49:13.500 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 16:49:18.502 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 16:49:38.511 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 16:49:43.512 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [67]:
l = sy.lib.python.list.List([2,3]).send(duet)

In [69]:
l[0]

2020-11-08 13:56:22.472 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:56:27.474 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:56:32.477 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:00:52.611 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:00:57.613 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [66]:
a

2

2020-11-08 13:55:27.439 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:55:32.443 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:56:07.464 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:56:12.466 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [43]:
a[0].shape

torch.Size([2, 20, 100])

2020-11-08 13:49:45.838 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:49:50.839 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:51:15.884 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:51:20.886 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [31]:
output

2020-11-08 13:46:54.739 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:46:59.741 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:47:04.742 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:47:09.745 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:47:14.747 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [ ]:
c = torch.ones(5,5)
c[torch.LongTensor([1,3])]

In [ ]:
torch.nn.init.xavier_uniform_(torch.empty(5,5), gain=1.0).send(duet)

_____ Local model test ______

In [25]:
import torch
import torch.nn as nn

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        self.ntoken = ntoken
        #self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        # Optionally tie weights as in:
        # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
        # https://arxiv.org/abs/1608.05859
        # and
        # "Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling" (Inan et al. 2016)
        # https://arxiv.org/abs/1611.01462
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        #self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.weight)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, x):
        input, hidden = x
        #emb = self.drop(self.encoder(input))
        emb = self.encoder(input)

        result = self.rnn(emb)
        output, hidden = result[0], result[1]
        
        #output = self.drop(output)
        decoded = self.decoder(output)
        decoded = decoded.view(-1, self.ntoken)
        
        #return F.log_softmax(decoded, dim=1)#, hidden
        return result

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)

2020-11-10 16:35:20.482 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 16:35:25.484 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [26]:
model = RNNModel(rnn_type = 'LSTM', ninp = 100, ntoken = 100, nhid = 100, nlayers =2)

2020-11-10 16:35:30.487 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [27]:
input = torch.ones(2,20, dtype = torch.long) # bsz * bptt
hidden = torch.zeros(2, 20, 100) # nb_layers * bptt * nhid
c = torch.zeros(2, 20, 100) # nb_layers * bptt * nhid

In [28]:
#output = model((input, hidden))
#output, hidden = model((input, [hidden, c]))
output= model((input, None))

2020-11-10 16:35:35.489 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [29]:
type(output)

tuple

2020-11-10 16:35:40.492 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 16:35:45.495 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 16:39:55.635 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 16:40:00.637 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


### Remaining issues

1. passing tuple of (hidden, c) to self.rnn gives exception in MergeFrom() call
2. Dropout layer does not seems to be working
3. when calling `input, hidden = self.rnn(input, None)` in LSTM case, we get two many values to unpack. This works in the local torch case. To solve the problem, I called it as `input = self.rnn(input, None)`
4. We cannot index a list pointer `l = syft.lib.python.list.List([1,2]).send(duet); a = l[0]`
5. We cannot unpack a list pointer `a,b = l`

In [3]:
import torch
t1 = torch.tensor([2,3,5])
t2 = torch.tensor([4,1,6])

list_ptr = sy.lib.python.list.List([t1,t2]).tag('#list').send(duet)
elem_ptr = list_ptr[0] #Does not work

In [4]:
duet.store.pandas

""


In [5]:
elem_ptr, list_ptr

(<syft.proxy.syft.lib.python.AnyPointer at 0x7f85c7a798d0>,
 <syft.proxy.syft.lib.python.ListPointer at 0x7f85c6100510>)

2020-11-10 15:59:51.325 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [6]:
l = elem_ptr.get(request_block=True)


> Waiting for Blocking Request
<UID:0a1e4b87-b199-48f4-8933-2889741838e3>

> INSIDE Request BLOCK 1.430511474609375e-06 seconds False
.
> INSIDE Request BLOCK 1.0820655822753906 seconds False

> Blocking Request ACCEPTED


In [7]:
l

tensor([2, 3, 5])

2020-11-10 15:59:56.327 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 16:00:01.330 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 16:01:06.364 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-10 16:01:11.368 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [30]:
t = duet.torch.Tensor(elem_ptr )

2020-11-08 20:19:30.843 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


In [32]:
t

2020-11-08 20:21:30.903 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-08 20:21:35.906 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-08 20:21:40.909 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


In [16]:
t = t + 3

2020-11-08 17:39:20.178 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [33]:
t = t.get(request_block = True)


> Waiting for Blocking Request
<UID:ea027da3-e0ca-43dc-8a96-dccb18e9e288>

> INSIDE Request BLOCK 1.1920928955078125e-06 seconds False


2020-11-08 20:21:45.911 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


2020-11-08 20:21:50.912 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


...

2020-11-08 22:18:14.516 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


2020-11-08 22:18:19.518 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


KeyboardInterrupt: 

In [18]:
type(t)

NoneType

2020-11-08 17:39:46.823 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 17:39:51.826 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

2020-11-08 20:06:10.340 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-08 20:06:15.342 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-08 20:06:20.344 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


In [78]:
t1_ptr, t2_ptr = list_ptr

ValueError: too many values to unpack (expected 2)

2020-11-08 14:10:42.889 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:10:47.891 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:14:58.018 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:15:03.020 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5
